In [1]:
def generateLabel(path):

    # Load the JSON data
    with open(path+"IR_label.json",'r') as load_f:
        json_data = json.load(load_f)

    # Get the bounding boxes
    boxes = json_data['gt_rect']
    exist = json_data['exist']

    # Define the image size
    image_width = 640
    image_height = 512

    # Loop over the bounding boxes and create a YOLOv7 formatted string for each one
    for i, box in enumerate(boxes):
        # Calculate the normalized coordinates and dimensions
        if box:
            x = (box[0] + box[2] / 2) / image_width
            y = (box[1] + box[3] / 2) / image_height
            width = box[2] / image_width
            height = box[3] / image_height

        # Create the YOLOv7 formatted string
        if (exist[i]==1):
            yolo_string = f"0 {x:.6f} {y:.6f} {width:.6f} {height:.6f}"
        else: yolo_string = f""

        number = "%06d" % (i+1)
        # Save the YOLOv7 formatted string to a file
        with open(path+f"{number}.txt", "w") as f:
            f.write(yolo_string)


In [2]:
import os
import json

#generate label txt file from json file
path = "./Origin_dataset/train/"
count=0
for video_name in os.listdir(path):
    if (not video_name.startswith('.')):
        video_path = os.path.join(path+video_name+"/")
        count+=1
        generateLabel(video_path)
print(count)
    

150


In [3]:
import shutil

# turn mixture into image and label two set
for video_name in os.listdir(path):
    if (not video_name.startswith('.')):
        video_path = os.path.join(path+video_name+"/")
        os.mkdir(os.path.join(video_path, "images"))
        os.mkdir(os.path.join(video_path, "labels"))
        for file_name in os.listdir(video_path):
            if file_name.endswith(".jpg"):
                shutil.move(os.path.join(video_path, file_name), os.path.join(video_path, "images", file_name))
            elif file_name.endswith(".txt"):
                shutil.move(os.path.join(video_path, file_name), os.path.join(video_path, "labels", file_name))



In [4]:
#choose a part of all 150 videos to form a mixed dataset

out = "./VOC/"
os.mkdir(os.path.join(out, "images"))
os.mkdir(os.path.join(out, "labels"))

interval= 100

for set_name in ["images", "labels"]:
    i=1
    for video_name in os.listdir(path):
        if (not video_name.startswith('.')):
            video_path = os.path.join(path+video_name+"/")
            category_path = os.path.join(video_path+set_name+"/")
            file_list = os.listdir(category_path)
            file_list.sort()
            
            count = len(file_list)
            for index in range(0,count,interval):
                file_name = file_list[index]
                if set_name=="images":
                    new_file_name = str(i)+".jpg"
                else:
                    new_file_name = str(i)+".txt"
                i+=1
                shutil.copy(os.path.join(category_path, file_name), os.path.join(out, set_name, new_file_name))

In [5]:
# split train and valid
train_ratio = 0.7
val_ratio = 0.3

for set_name in ["images", "labels"]:
    os.mkdir(os.path.join(out, set_name, "train"))
    os.mkdir(os.path.join(out, set_name, "valid"))
    os.mkdir(os.path.join(out, set_name, "test"))


from sklearn.model_selection import train_test_split

image_folder = out+"images"
label_folder = out+"labels"

# Get list of image filenames without the extension
image_filenames = [os.path.splitext(filename)[0] for filename in os.listdir(image_folder)]

# Get corresponding list of label filenames
label_filenames = [os.path.join(label_folder, f"{filename}.txt") for filename in image_filenames]

# Combine the two lists into a list of tuples
data = list(zip(image_filenames, label_filenames))

# Split the data into training and testing sets
train_data, valid_data = train_test_split(data, test_size=0.3, random_state=42)


# Extract the image and label filenames from the training set
for number, train_label_path in train_data:
    if number.isdigit():
        shutil.move(os.path.join(out, "images", number+".jpg"), os.path.join(out, "images", "train", number+".jpg"))
        shutil.move(train_label_path, os.path.join(out, "labels", "train", str(number)+".txt"))

# Extract the image and label filenames from the testing set
for number, val_label_path in valid_data:
    if number.isdigit():
        shutil.move(os.path.join(out, "images", number+".jpg"), os.path.join(out, "images", "valid", number+".jpg"))
        shutil.move(val_label_path, os.path.join(out, "labels", "valid", str(number)+".txt"))


